In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
classes = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}

## Loading Data

In [10]:
master_data=[]
for i in ["train", "test"]:
    splity=[]
    p1="/Users/pranavthakkar/Desktop/srip/UCI HAR Dataset/"+i+"/Inertial Signals"
    # print(p1)
    for s_type in ['body_acc', 'body_gyro', 'total_acc']:
        p2=p1+"/"+s_type
        # print(p2)
        for axis in ['x', 'y', 'z']:
            file_path = p2 + f"_{axis}_{i}.txt"
            # print(file_path)
            d=np.loadtxt(file_path).reshape(-1, 128, 1)
            # print(d.shape)
            splity.append(d)
    split_data=np.concatenate(splity, axis=2)
    master_data.append(split_data)

X_train=master_data[0]
X_test=master_data[1]            



In [11]:
X_train.shape, X_test.shape

((7352, 128, 9), (2947, 128, 9))

## Loading labels data

In [12]:
y_test = np.loadtxt("/Users/pranavthakkar/Desktop/srip/UCI HAR Dataset/test/y_test.txt").astype(int) 
y_train = np.loadtxt("/Users/pranavthakkar/Desktop/srip/UCI HAR Dataset/train/y_train.txt").astype(int) 

In [13]:
y_train.shape, y_test.shape

((7352,), (2947,))

## 1D CNN

In [14]:
m_cnn = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(128, 9)),  
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(), #for converting 2D to 1D for dense layer.
    Dense(64, activation='relu'),
    # Dense(24, activation='relu'),
    Dense(6, activation='softmax')  # 6 classes
])
m_cnn.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])
m_cnn.summary()

/Users/pranavthakkar/Documents/GitHub/Srip_Pre-Req/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 126, 64)        │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 63, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 61, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3840)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       245,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 272,710 (1.04 MB)

 Trainable params: 272,710 (1.04 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# -1 as the classes are 1-6 and we need 0-5 for sparse categorical crossentropy.
training_cnn = m_cnn.fit(X_train, y_train-1, epochs=15, batch_size=32,validation_data=(X_test, y_test-1))

Epoch 1/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7202 - loss: 0.6862 - val_accuracy: 0.8677 - val_loss: 0.3999
Epoch 2/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9398 - loss: 0.1597 - val_accuracy: 0.8945 - val_loss: 0.3672
Epoch 3/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9482 - loss: 0.1246 - val_accuracy: 0.8755 - val_loss: 0.3953
Epoch 4/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9479 - loss: 0.1220 - val_accuracy: 0.9050 - val_loss: 0.3780
Epoch 5/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9570 - loss: 0.1009 - val_accuracy: 0.8972 - val_loss: 0.4414
Epoch 6/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9591 - loss: 0.0958 - val_accuracy: 0.9040 - val_loss: 0.4324
Epoch 7/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9579 - loss: 0.0942 - val_accuracy: 0.9030 - val_loss: 0.4505
Epoch 8/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9627 - loss: 0.0846 - val_accuracy: 0.

In [16]:
print("Train Accuracy:", training_cnn.history['accuracy'][-1])
print("Test Accuracy:", training_cnn.history['val_accuracy'][-1])

Train Accuracy: 0.9723884463310242
Test Accuracy: 0.9155073165893555


## LSTM

In [17]:
m_lstm = Sequential([
    LSTM(84, return_sequences=True, input_shape=(128, 9)),  # First LSTM layer
    LSTM(42),  # Second LSTM layer
    Dense(18, activation='relu'),
    Dense(6, activation='softmax')  # Output layer for 6 classes
])
m_lstm.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])
m_lstm.summary()

/Users/pranavthakkar/Documents/GitHub/Srip_Pre-Req/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128, 84)        │        31,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 42)             │        21,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 18)             │           774 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           114 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,808 (210.19 KB)

 Trainable params: 53,808 (210.19 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# -1 as the classes are 1-6 and we need 0-5 for sparse categorical crossentropy.
training_lstm = m_lstm.fit(X_train, y_train-1, epochs=15, batch_size=32,validation_data=(X_test, y_test-1))

Epoch 1/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 16s 66ms/step - accuracy: 0.4641 - loss: 1.3177 - val_accuracy: 0.6783 - val_loss: 0.8166
Epoch 2/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 15s 64ms/step - accuracy: 0.7200 - loss: 0.6692 - val_accuracy: 0.7292 - val_loss: 0.6570
Epoch 3/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 15s 63ms/step - accuracy: 0.7657 - loss: 0.5610 - val_accuracy: 0.7374 - val_loss: 0.6716
Epoch 4/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.8086 - loss: 0.4706 - val_accuracy: 0.7560 - val_loss: 0.6798
Epoch 5/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 15s 64ms/step - accuracy: 0.8906 - loss: 0.3126 - val_accuracy: 0.8571 - val_loss: 0.4117
Epoch 6/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 14s 63ms/step - accuracy: 0.9378 - loss: 0.1721 - val_accuracy: 0.8775 - val_loss: 0.4083
Epoch 7/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 15s 64ms/step - accuracy: 0.9441 - loss: 0.1567 - val_accuracy: 0.8524 - val_loss: 0.4347
Epoch 8/15
230/230 ━━━━━━━━━━━━━━━━━━━━ 15s 64ms/step - accuracy: 0.9455 - loss: 0.1384 - 

In [19]:
print("Train Accuracy:", training_lstm.history['accuracy'][-1])
print("Test Accuracy:", training_lstm.history['val_accuracy'][-1])

Train Accuracy: 0.9528019428253174
Test Accuracy: 0.903630793094635


## Summary

We trained 2 Deep Learning models : 1D-CNN and LSTM on the raw acceleeration data.
And the 1D-CNN performed better in both Training and test accuracy. Also, training of 1D-CNN was much faster compared to LSTM.
